In [1]:
import elasticsearch
import unicodedata
import csv
from elasticsearch_dsl.aggs import Terms, Nested
from elasticsearch_dsl.search import Search
from elasticsearch_dsl import Q
import networkx as nx
import pandas as pd
import pickle
import datetime
from pandasticsearch import Select
import numpy as np

es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")

#####################################################################
# Code for quering between dates (Not Working)
#####################################################################
# field_query = Q('query_string', query='_exists_:ept.runtime_seconds')
# time_query = Q('range',**{"@timestamp":{
#     'from': '2018-10-24T00:00:00.000Z',
#     'to': '2018-10-24T23:59:59.999Z'
# }})
# combined = Q('bool', filter=[field_query, time_query])
# res = Search(using=es, index="packetbeat*")
# response = res.query(combined)
#####################################################################


#####################################################################
# Make your Query with elasticsearch_dsl
#####################################################################
#res = Search(using=es, index="packetbeat*").filter('range',**{"@timestamp":{'from': datetime.datetime.now().strftime('2018-10-23T00:00:00.000Z'), 'to' : datetime.datetime.now().strf
#time('2019-01-31T23:59:59.999Z')}})
res = Search(using=es, index="packetbeat*").filter('range',**{"@timestamp":{'gte': '2018-10-27T00:00:00.000Z', 'lt' : '2018-10-27T23:59:59.999Z'}}).params(request_timeout=10000)#.sort("timestamp", {'order': "desc"})
#res.aggs.bucket('properties', Nested(path='properties')).bucket('hostname', Terms(field='host.name.keyword'))
# res.sort('@timestamp')
#####################################################################
# Print the underlying JSON representation of your Query
#####################################################################
res_json = res.to_dict()
print(res_json)
    
#####################################################################
# Syntax for filtering data
#####################################################################
#        .query('bool', filter=Q('exists', field='ip') & Q('exists', field='port')\
#               & Q('exists', field='client_ip') & Q('exists', field='client_port')\
#               & Q('exists', field='type') & Q('exists', field='method') & Q('exists', field='status')\
#               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')
#              )
# Code for quering between dates (Not Working)
#####################################################################

#    .query("match", ip="/[0-9]{2,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}/")

response = res.execute()
#filtered=res.filter("exists", field='ip' & "exists", field='client_ip').count()
#print(filtered)

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

#h = response.hits[0]
#print('/%s/%s/%s returned with score %f' % (
#    h.meta.index, h.meta.doc_type, h.meta.id, h.meta.score))

#df = pd.DataFrame((d.to_dict() for d in res))
###print(res['hits']['hits'])

{'query': {'bool': {'filter': [{'range': {'@timestamp': {'gte': '2018-10-27T00:00:00.000Z', 'lt': '2018-10-27T23:59:59.999Z'}}}]}}}
Filtered logs: 4723499
Total number of logs: 4723499


In [2]:
#####################################################################
# Make a small test for normalizing a json response (single flow) from elasticsearch
#####################################################################
from pandas.io.json import json_normalize
my_json = {'flow_id': 'EQQA////DP//////FP8BAAH6Fj4hHCz6Fj5kj2kKAQFBCgEBGLQTxuw', 'host': {'name': 'elk-master'}, 'start_time': '2018-10-23T23:47:34.605Z', '@timestamp': '2018-10-23T23:48:20.193Z', 'type': 'flow', 'source': {'ip': '10.1.1.24', 'port': 60614, 'stats': {'net_bytes_total': 14750, 'net_packets_total': 48}, 'mac': 'fa:16:3e:64:8f:69'}, 'final': False, 'tags': ['beats_input_raw_event'], 'dest': {'ip': '10.1.1.65', 'port': 5044, 'stats': {'net_bytes_total': 2424, 'net_packets_total': 24}, 'mac': 'fa:16:3e:21:1c:2c'}, 'beat': {'hostname': 'elk-master', 'name': 'elk-master', 'version': '6.4.2'}, 'transport': 'tcp', '@version': '1', 'last_time': '2018-10-23T23:48:17.797Z'}
pandas_df = json_normalize(my_json)
#pandas_df.head()


In [3]:
pandas_df.head()
print(datetime.datetime.now())
# n_rows = len(pandas_df.index)
# if (n_rows % 100000) == 0:
#     print(n_rows)

2019-06-30 03:38:35.459101


In [4]:
#pandas_df = Select.from_dict(res.scan()).to_pandas()
#df = pd.concat(map(pd.DataFrame.from_dict, res.scan()), axis=1)['fields'].T
# from pandas.io.json import json_normalize
# df = json_normalize(res['hits']['hits'])


# for hits in res.scan():
#     print(hits.to_dict())
#     print("\n")

# for hits in res.scan():
#     pandas_df2 = json_normalize(hits.to_dict())

# pandas_df2.head
# print(res.scan()['hits']['hits'])

#####################################################################
# Create a dummy dataframe to which we will append flows from Elasticsearch
#####################################################################
todays_date = datetime.datetime.now().date()
index = index = pd.date_range(todays_date-datetime.timedelta(1), periods=1, freq='D')
# index = 0
data = np.array([np.arange(1)]*3).T
# data = ['dummy','dummy','dummy']
columns = ['@timestamp','beat.hostname', 'beat.version']
main_pandas_df = pd.DataFrame(data, index=index, columns=columns)

#pandas_df = Select.from_dict(x for hit in res.scan()).to_pandas()
#main_pandas_df

#####################################################################
# CNormalize each response (flow in response) and append to dataframe
#####################################################################
print("#####################")
print(datetime.datetime.now())
print("#####################")
for hits in res.scan():
    temp_df = json_normalize(hits.to_dict())
    #main_pandas_df = pd.concat([main_pandas_df,temp_df],axis=1)
    main_pandas_df = main_pandas_df.append(temp_df, ignore_index=True, sort=False)
    n_rows = len(main_pandas_df.index)
    if (n_rows % 2500) == 0:
        print(datetime.datetime.now())
        print(n_rows)
        print("\n")
    
#####################################################################
# Visualize the results
#####################################################################
#main_pandas_df.head(50)

#####################
2019-06-30 03:38:37.249955
#####################
2019-06-30 03:39:24.541809
2500


2019-06-30 03:40:15.112595
5000


2019-06-30 03:41:24.548983
7500


2019-06-30 03:42:51.426345
10000


2019-06-30 03:44:37.141480
12500


2019-06-30 03:46:43.380291
15000


2019-06-30 03:49:12.275193
17500


2019-06-30 03:52:03.940104
20000


2019-06-30 03:55:20.210402
22500


2019-06-30 03:59:05.069492
25000


2019-06-30 04:03:16.338932
27500


2019-06-30 04:07:50.617647
30000


2019-06-30 04:12:50.603592
32500


2019-06-30 04:18:14.327096
35000


2019-06-30 04:24:01.872780
37500


2019-06-30 04:30:12.366791
40000


2019-06-30 04:36:48.170087
42500


2019-06-30 04:43:48.383488
45000


2019-06-30 04:51:14.348073
47500


2019-06-30 04:59:09.430513
50000


2019-06-30 05:07:31.970271
52500


2019-06-30 05:16:21.459223
55000


2019-06-30 05:25:41.806746
57500


2019-06-30 05:35:30.184269
60000


2019-06-30 05:45:49.737649
62500


2019-06-30 05:56:36.307933
65000


2019-06-30 06:07:53.65

ScanError: Scroll request has only succeeded on 4 shards out of 5.

In [ ]:
######################################################################
# Do NOT Delete - Extracting specific fields from ES response
######################################################################

# df_tcp_udp = pd.DataFrame(((hit["@timestamp"],hit['host']['name'],hit['type'],hit['method'],hit['path'],hit['direction'],hit['status'],hit['ip'],hit['port'],hit['client_ip'],hit['client_port'],hit['bytes_in'],hit['bytes_out'],hit['responsetime'],hit['http.request.headers.content-length'],hit['http.request.params'],hit['http.request.code'],hit['http.response.headers.content-length'],hit['http.response.headers.content-type'],hit['http.response.phrase'], hit['dest.stats.net_bytes_total'],hit['dest.stats.net_packets_total'],hit['source.stats.net_bytes_total'],hit['source.stats.net_packets_total'],hit['final']) for hit in res.scan()),\
#                     columns=['Timestamp','hostname','type','method','path','direction','status','ip','port','client_ip','client_port','bytes_in','bytes_out','responsetime','http.request.headers.content-length','http.request.params','http.response.code','http.response.headers.content-length','http.response.headers.content-type','http.response.phrase','dest.stats.net_bytes_total','dest.stats.net_packets_total','source.stats.net_bytes_total','source.stats.net_packets_total','final'])

######################################################################
# Sorting by timestamp using this approach did not worked
######################################################################
# Convert timestamp to datetime
main_pandas_df['@timestamp'] = pd.to_datetime(main_pandas_df['@timestamp'])
# Check the number of fields and the data type for each field
#print(main_pandas_df.info())                        
# Sorting by the timestamp column didn't work
#main_pandas_df.sort_values(by=['@timestamp'])
# Convert timestamp to datetime - Use only in case of not running the previous cell
main_pandas_df['@timestamp'] = pd.to_datetime(main_pandas_df['@timestamp'])
#Set timestamp as the index - the timestamp column will not be deleted
main_pandas_df.index = main_pandas_df['@timestamp']
#Sort by Timestamp
main_pandas_df.sort_index(inplace=True)
#Select specific columns to show
filtered_result = main_pandas_df[['@timestamp','dest.stats.net_bytes_total','flow_id']]
#print(main_filter_id.info())
filtered_result.head()

# Print first 10 records
main_pandas_df.head(10)
# filtered = main_pandas_df[(main_pandas_df.type == 'http')]
# filtered.head(10)



In [ ]:
print(main_pandas_df.info())

In [ ]:
######################################################################
# Increase the number of displayed results
######################################################################
###pd.set_option('display.max_rows', 500)
###pd.set_option('display.max_colwidth', -1)
######################################################################
# Sorting flows by timestamp requires indexing the timestamp column
######################################################################
# Filtering for a specific flod_id
###filter_id = main_pandas_df.flow_id == 'EQQA////DP//////FP8BAAH6Fj4hHCz6Fj5QykYKAQFBCgEBRxriVCQ'
###main_filter_id = main_pandas_df[(filter_id)]
# Convert timestamp to datetime - Use only in case of not running the previous cell
###main_filter_id['@timestamp'] = pd.to_datetime(main_filter_id['@timestamp'])
# main_filter_id['@timestamp']#.dt.tz_convert('US/Central')
#main_filter_id['@timestamp'].astimezone(timezone('US/Pacific'))
# Sorting by the timestamp column didn't work
#main_filter_id.sort_values(by=['@timestamp'])
#Set timestamp as the index - the timestamp column will not be deleted
###main_filter_id.index = main_filter_id['@timestamp']
#main_filter_id.index.tz_convert('Europe/Berlin')
#Sort by Timestamp
###main_filter_id.sort_index(inplace=True)
#Select specific columns to show
###filtered_result = main_filter_id[['@timestamp','dest.stats.net_bytes_total','flow_id']]
#print(main_filter_id.info())
###filtered_result
#filtered_result2 = main_pandas_df[['@timestamp','dest.stats.net_bytes_total','flow_id']]
#filtered_result2

In [ ]:
#main_pandas_df.to_pickle("/mnt/sdb/tcp_udp_dataframe.pkl")
main_pandas_df.to_pickle("/mnt/sdb/netflows_2018-10-26_2018-10-30.pkl")

In [ ]:
es = elasticsearch.Elasticsearch(["149.165.170.209:9200"])
#res = Search(using=es, index="filebeat*").query("match", username="root")
res = Search(using=es, index="packetbeat*")\
        .query('bool', filter=Q('exists', field='ip')\
               & Q('exists', field='client_ip')\
               & Q('exists', field='type') & Q('exists', field='status')\
               & Q('exists', field='bytes_in') & Q('exists', field='bytes_out')\
               & Q('match', type='ICMP')
              )

response = res.execute()

print("Filtered logs: %i" %res.count()) #There is an error on the number of matched logs
print("Total number of logs: %i" %response.hits.total)

In [ ]:
df_icmp = pd.DataFrame(((hit["@timestamp"],hit['type'],hit['status'],hit['ip'],hit['client_ip'],hit['bytes_in'],hit['bytes_out']) for hit in res.scan()),\
                    columns=['Timestamp','type','status','ip','client_ip','bytes_in','bytes_out'])


In [ ]:
df_icmp.to_pickle("icmp_dataframe.pkl")

In [ ]:
df_icmp[:-10]

In [ ]:
df_tcp_udp[:-10]